# Safety Stock Example




In [1]:
# Install Required Packages
# %pip install optiguide

In [2]:
# test Gurobi installation
import gurobipy as gp
from gurobipy import GRB
from eventlet.timeout import Timeout

# import auxillary packages
import requests  # for loading the example source code
import openai

# import flaml and autogen
from flaml import autogen
from flaml.autogen.agentchat import Agent, UserProxyAgent
from optiguide.optiguide import OptiGuideAgent

import os

In [3]:
autogen.oai.ChatCompletion.start_logging()

config_list = autogen.config_list_from_json(
    "OAI_CONFIG_LIST",
    filter_dict={
        "model": {
            "gpt-4",
            "gpt4",
            "gpt-4-32k",
            "gpt-4-32k-0314",
            "gpt-3.5-turbo",
            "gpt-3.5-turbo-16k",
            "gpt-3.5-turbo-0301",
            "chatgpt-35-turbo-0301",
            "gpt-35-turbo-v0301",
        }
    },
)
openai.api_key = os.environ["OPENAI_API_KEY"]

Now, let's import the source code (loading from URL) and also some training examples (defined as string blow).

In [4]:
# Get the source code of our coffee example
# code_url = "https://raw.githubusercontent.com/yuqingsh/capstone-project-demo/main/safety_stock.py"
# response = requests.get(code_url)


# # Check if the request was successful
# if response.status_code == 200:
#     # Get the text content from the response
#     code = response.text
# else:
#     raise RuntimeError("Failed to retrieve the file.")
code = open('safety_stock.py', "r").read() # for local files


# show the first head and tail of the source code
print("\n".join(code.split("\n")[:10]))
print(".\n" * 3)
print("\n".join(code.split("\n")[-10:]))

from gurobipy import Model, GRB
import pandas as pd
import time


# Calculate statistics
df = pd.read_csv("data/supply_chain_data.csv")
average_demand = df["Number of products sold"].mean()  # average demand per period
demand_std_dev = df["Number of products sold"].std()  # standard deviation of demand
lead_time = 3  # lead time in periods
.
.
.

model.optimize()

print(time.ctime())

# Print the optimal safety stock
if m.status == GRB.OPTIMAL:
    print(f"Optimal Safety Stock: {safety_stock.x}")
else:
    print("No optimal solution found")



In [5]:
# In-context learning examples.
example_qa = """
----------
Question: What is the 95 percent safety stock of the products?
Answer Code:
```python
z_score = 1.65
```

----------
Question: Can you make a more conservative estimation?
Answer Code:
```python
z_score = 1.96
```

----------
Question: What's the safety stock?
Answer Code:
```python
z_score = 1.96
```
"""

Now, let's create an OptiGuide agent and also a user.

For the OptiGuide agent, we only allow "debug_times" to be 1, which means it can debug its answer once if it encountered errors.

In [6]:
%%capture
agent = OptiGuideAgent(
    name="Safety Stock",
    source_code=code,
    debug_times=1,
    example_qa="",
    llm_config={
        "request_timeout": 600,
        "seed": 42,
        "config_list": config_list,
    }
)

user = UserProxyAgent(
    "user", max_consecutive_auto_reply=0,
    human_input_mode="NEVER", code_execution_config=False
)

Now, let's create a user's question.

In [7]:
user.initiate_chat(
    agent, message="Can you calculate the safety stock for our products?"
)

user (to Safety Stock):

Can you calculate the safety stock for our products?

--------------------------------------------------------------------------------
Safety Stock (to writer):


Answer Code:


--------------------------------------------------------------------------------
writer (to Safety Stock):

```python
import scipy.stats as stats

# Using the historical demand and standard deviation, calculate safety stock
safety_stock = demand_std_dev * stats.norm.ppf(service_level) + average_demand * lead_time
safety_stock
```

--------------------------------------------------------------------------------
Safety Stock (to safeguard):


--- Code ---
import scipy.stats as stats

# Using the historical demand and standard deviation, calculate safety stock
safety_stock = demand_std_dev * stats.norm.ppf(service_level) + average_demand * lead_time
safety_stock

--- One-Word Answer: SAFE or DANGER ---


--------------------------------------------------------------------------------
safeg